<a href="https://colab.research.google.com/github/ExplorerLGD/StupidAI/blob/master/MYDQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T



# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [0]:
# Hyper Parameters
BATCH_SIZE = 128
LR = 0.01                   # learning rate
EPSILON = 0.9               # greedy policy

TARGET_REPLACE_ITER = 100   # target update frequency
MEMORY_CAPACITY = 2000

GAMMA = 0.9
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

class Net(nn.Module):
  def __init__(self,):
    super(Net,self).__init__()
    self.fc1=nn.Linear(1,50)
    self.fc1.weight.data.normal_(0,0.1)
    self.out=nn.Linear(50,1)
    self.fc1.weight.data.normal_(0,0.1)
  def forward(self,x):
    x=self.fc1(x)
    x=F.relu(x)
    actions_value=self.out(x)
    return actions_value
  

policy_net,target_net=Net(),Net()
steps_done=0
memory = ReplayMemory(10000)
optimizer = torch.optim.Adam(policy_net.parameters(), lr=LR)
loss_func = nn.MSELoss()
n_actions =2

def select_action(state):
  global steps_done
  sample=random.random()
  eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
  steps_done += 1
  if sample > eps_threshold:
      with torch.no_grad():
          # t.max(1) will return largest column value of each row.
          # second column on max result is index of where max element was
          # found, so we pick action with the larger expected reward.
          return policy_net(state).max(1)[1].view(1, 1)
  else:
      return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)  
    
def optimize_model():
  if len(memory)<BATCH_SIZE:
    return
  transitions=memory.sample(BATCH_SIZE)
  batch = Transition(*zip(*transitions))
  
  non_final_mask=torch.tensor(tuple(map(lambda s:s is not None,batch.next_state)),device=device,dtype=torch.uint8)
  non_final_next_states=torch.cat([s for s in batch.next_state if s is not None])
  state_batch=torch.cat(batch.state)
  action_batch=torch.cat(batch.action)
  reward_batch=torch.cat(batch.reward)
  
  state_action_values=policy_net(state_batch).gather(1,action_batch)
  next_state_value=torch.zeros(BATCH_SIZE,device=device)
  next_state_value[non_final_mask]=target_net(non_final_next_states).max(1)[0].detach()
  expected_state_action_values = (next_state_values * GAMMA) + reward_batch
  
  loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
  
  optimizer.zero_grad()
  loss.backward()
  for param in policy_net.parameters():
    param.grad.data.clamp_(-1,1)
  optimizer.step()

In [0]:
class Env():
  def __init__(self,):
    self.num=0
    
  def get_num(self):
    num=torch.randint(0,100,[1])
    self.num=num
    return num
  def step(self,x):
    result=abs(x-self.num)
    done=0
    info=0
    return result,result,done,info



In [0]:
env=Env()
num_episodes = 50

for i_episode in range(num_episodes):
    # Initialize the environment and state
    #env.reset()
    #state=(编不下去了)
    for t in count():
        # Select and perform an action
        action = select_action(state)
        _, reward, done, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)

        # Observe new state
        last_screen = current_screen
        current_screen = get_screen()
        if not done:
            next_state = current_screen - last_screen
        else:
            next_state = None

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)

            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')


NameError: ignored

In [0]:
test=torch.tensor([2,3,2,4])
gather=test.gather(0,torch.tensor([0,1]))
print(gather)
